In [1]:
import tensorflow as tf
import numpy as np
TRAINING = True
tf.reset_default_graph()
def identity_block(X_input, kernel_size, filters, stage, block, relu):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    with tf.name_scope("id_block_stage"+str(stage)):
        filter1, filter2, filter3 = filters
        X_shortcut = X_input

        # First component of main path
        x = tf.layers.conv2d(X_input, filter1,
                 kernel_size=(1, 1), strides=(1, 1),name=conv_name_base+'2a')
        x = tf.layers.batch_normalization(x, axis=3, name=bn_name_base+'2a', training=TRAINING)
        x = tf.nn.relu(x)

        # Second component of main path
        x = tf.layers.conv2d(x, filter2, (kernel_size, kernel_size),
                                 padding='same', name=conv_name_base+'2b')
        x = batch_norm2 = tf.layers.batch_normalization(x, axis=3, name=bn_name_base+'2b', training=TRAINING)
        x = tf.nn.relu(x)

        # Third component of main path
        x = tf.layers.conv2d(x, filter3, kernel_size=(1, 1),name=conv_name_base+'2c')
        x = tf.layers.batch_normalization(x, axis=3, name=bn_name_base + '2c', training=TRAINING)

        # Final step: Add shortcut value to main path, and pass it through a RELU activation
        X_add_shortcut = tf.add(x, X_shortcut)
        if relu==True:
            add_result = tf.nn.relu(X_add_shortcut)
        else:
            add_result=X_add_shortcut
    return add_result

def convolutional_block(X_input, kernel_size, filters, stage, block, stride, relu):
    #change the shape of output so that it can do sum process with shotcut
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    with tf.name_scope("conv_block_stage" + str(stage)):

        # Retrieve Filters
        filter1, filter2, filter3 = filters

        # Save the input value
        X_shortcut = X_input

        # First component of main path
        x = tf.layers.conv2d(X_input, filter1,
                                 kernel_size=(1, 1),
                                 strides=(stride, stride),
                                 name=conv_name_base+'2a')
        x = tf.layers.batch_normalization(x, axis=3, name=bn_name_base+'2a', training=TRAINING)
        x = tf.nn.relu(x)

        # Second component of main path
        x = tf.layers.conv2d(x, filter2, (kernel_size, kernel_size), name=conv_name_base + '2b',padding='same')
        x = tf.layers.batch_normalization(x, axis=3, name=bn_name_base + '2b', training=TRAINING)
        x = tf.nn.relu(x)

        # Third component of main path
        x = tf.layers.conv2d(x, filter3, (1, 1), name=conv_name_base + '2c')
        x = tf.layers.batch_normalization(x, axis=3, name=bn_name_base + '2c', training=TRAINING)

        # SHORTCUT PATH
        X_shortcut = tf.layers.conv2d(X_shortcut, filter3, (1,1),
                                      strides=(stride, stride), name=conv_name_base + '1')
        X_shortcut = tf.layers.batch_normalization(X_shortcut, axis=3, name=bn_name_base + '1', training=TRAINING)

        # Final step: Add shortcut value to main path, and pass it through a RELU activation
        X_add_shortcut = tf.add(X_shortcut, x)
        if relu==True:
            add_result = tf.nn.relu(X_add_shortcut)
        else:
            add_result = X_add_shortcut
    return add_result

def attention_block(X_input, channel_output, attention_name, p=1, t=2, r=1):
    def upsample(x_t, name, size):
        with tf.name_scope(name):
            outputs = tf.image.resize_bilinear(x_t, size)
            return outputs
        
    with tf.name_scope(attention_name+'pre_processing'):
        pre_pros = convolutional_block(X_input, 3, [channel_output/4, channel_output/4, channel_output], 
                                       stage=attention_name, block='pre_proc', stride=1, relu=False)
    
    with tf.name_scope(attention_name+'trunk_branch'):
        for idx in range(t):
            idx=str(idx)
            trunks = convolutional_block(pre_pros,3, [channel_output/4, channel_output/4, channel_output], 
                                       stage=attention_name, block='trunk'+idx, stride=1, relu=False)
    with tf.name_scope(attention_name+'mask_branch'):
        size_1 = pre_pros.get_shape().as_list()[1:3]
        mask_1 = tf.layers.max_pooling2d(pre_pros, pool_size=(3, 3),strides=(2, 2),name='pool_mask_1')
        for idx in range(r):
            idx=str(idx)
            mask_1 = convolutional_block(mask_1, 3, [channel_output/4, channel_output/4, channel_output], 
                                       stage=attention_name, block='mask_1'+idx, stride=1, relu=False)
        size_2 = mask_1.get_shape().as_list()[1:3]
        mask_2 = tf.layers.max_pooling2d(mask_1, pool_size=(3, 3),strides=(2, 2),name='pool_mask_2')
        for idx in range(2*r):
            idx=str(idx)
            mask_2 = convolutional_block(mask_2, 3, [channel_output/4, channel_output/4, channel_output], 
                                       stage=attention_name, block='mask_2'+idx, stride=1, relu=False)
        mask_3 =  upsample(mask_2,'upsample_1',size_2)
        mask_3 = tf.add(mask_3, mask_1, name="fuse_add")
        for idx in range(r):
            idx=str(idx)
            mask_3 = convolutional_block(mask_3, 3, [channel_output/4, channel_output/4, channel_output], 
                                       stage=attention_name, block='mask_3'+idx, stride=1, relu=False)
        mask_4 = upsample(mask_3,'upsample_2',size_1)
        mask_4 = convolutional_block(mask_4, 3, [channel_output/4, channel_output/4, channel_output], 
                                       stage=attention_name, block='mask_4', stride=1, relu=True)
        mask_4 = tf.nn.sigmoid(mask_4, "mask_sigmoid")
    with tf.name_scope(attention_name+'fusing'):
        x = tf.multiply(trunks, mask_4, name='fus_mul')
        x = tf.add(x, trunks, name='fus_add')
    with tf.name_scope(attention_name+'post_processing'):
        for idx in range(p):
            idx=str(idx)
            output = convolutional_block(x, 3, [channel_output/4, channel_output/4, channel_output], 
                                       stage=attention_name, block='post_pros', stride=1, relu=True)
    return output

In [2]:
def res_attention_inference(X_input, class_num=2):
    X = X_input
    x = tf.layers.conv2d(X, filters=32, kernel_size=(3, 3), strides=(1, 1), name='pre_cov')
    #64
    x = tf.layers.batch_normalization(x, axis=3, name='pre_bn_conv1')
    x = attention_block(x, 32, 'attention_1')
    #64
    x = convolutional_block(x, 3, [8,8,32],stage='trans', block='1', stride=1, relu=False)
    #64
    x = attention_block(x, 32, 'attention_2')
    #64
#     x = convolutional_block(x, 3, [8,8,32],stage='trans', block='2', stride=1, relu=False)
#     #64
#     x = attention_block(x, 32, 'attention_3')
    #64
    x = convolutional_block(x, 3, [64, 64, 256],stage='pos_pros', block='1', stride=2, relu=False)
    #32
#     x = convolutional_block(x, kernel_size=3, filters=[64, 64, 256], stage='resnet1', block='a', stride=2, relu=True)
#     x = identity_block(x, 3, [64, 64, 256], stage='resnet1', block='b', relu=True)
#     x = identity_block(x, 3, [64, 64, 256], stage='resnet1', block='c', relu=True)
    #16
    x = convolutional_block(x, kernel_size=3, filters=[256, 256, 1024], stage='resnet2', block='a', stride=1, relu=True)
    x = identity_block(x, 3, [256, 256, 1024], stage='resnet2', block='b', relu=True)
    x = identity_block(x, 3, [256, 256, 1024], stage='resnet2', block='c', relu=True)
#     x = identity_block(x, 3, [256, 256, 1024], stage='resnet2', block='d', relu=True)
#     x = identity_block(x, 3, [256, 256, 1024], stage='resnet2', block='e', relu=True)
#     x = identity_block(x, 3, [256, 256, 1024], stage='resnet2', block='f', relu=True)
    #16
    #x = tf.nn.lrn(x, depth_radius=4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm2')
    x = tf.layers.average_pooling2d(x, pool_size=(2, 2), strides=(1,1))
    flatten = tf.layers.flatten(x, name='flatten')
    keep_prob = 0.5
    # dropout
    drop = tf.nn.dropout(flatten, keep_prob)
    dense1 = tf.layers.dense(drop, units=50, activation=tf.nn.relu)
    logits = tf.layers.dense(dense1, units=2, activation=tf.nn.softmax)
    return logits

In [3]:
def losses(logits, labels, name):
    with tf.variable_scope('loss') as scope:
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits \
            (logits=logits, labels=labels, name='xentropy_per_example')
        loss = tf.reduce_mean(cross_entropy, name='loss')
        if name=='train':
            tf.summary.scalar(scope.name + '/train_loss', loss)
        if name=='valid':
            tf.summary.scalar(scope.name + '/valid_loss', loss)
    return loss
 
def trainning(loss, learning_rate):
    with tf.name_scope('optimizer'):
        optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)
        global_step = tf.Variable(0, name='global_step', trainable=False)
        train_op = optimizer.minimize(loss, global_step= global_step)
    return train_op
 
def evaluation(logits, labels, name):
    with tf.variable_scope('accuracy') as scope:
        correct = tf.nn.in_top_k(logits, labels, 1)
        correct = tf.cast(correct, tf.float16)
        accuracy = tf.reduce_mean(correct)
        if name=='train':
            tf.summary.scalar(scope.name + '/train_accuracy', accuracy)
        if name=='valid':
            tf.summary.scalar(scope.name + '/valid_accuracy', accuracy)
    return accuracy

def recall_precision(logits, labels, name):
    logits = tf.cast(logits, tf.int64)
    labels = tf.cast(labels, tf.int64)
    predict = tf.arg_max(logits,1)
    with tf.variable_scope('recall_precision') as scope:
        TP = tf.count_nonzero(predict * labels)
        TN = tf.count_nonzero((predict - 1) * (labels - 1))
        FN = tf.count_nonzero(predict * (labels - 1))
        FP = tf.count_nonzero((predict - 1) * labels)
        precision = tf.divide(TP, TP + FP)
        recall = tf.divide(TP, TP + FN)
        precision = tf.cast(precision, dtype=tf.float64)
        recall = tf.cast(recall, dtype=tf.float64)
        #f1 = 2 * precision * recall / (precision + recall)
        #f1 = tf.cast(f1, dtype=tf.float32)
        if name=='train':
            tf.summary.scalar(scope.name + '/train_precision', precision)
            tf.summary.scalar(scope.name + '/train_recall', recall)
        if name=='valid':
            tf.summary.scalar(scope.name + '/valid_precision', precision)
            tf.summary.scalar(scope.name + '/valid_recall', recall)
    return precision, recall

In [4]:
import os
 
def get_files(path_pos,path_neg,label_pos,label_neg):
    TC = []
    label_TC = []
    nonTC = []
    label_nonTC = []
    # data loader
    file_dir_TC=path_pos
    file_dir_nonTC=path_neg
    TC_list = os.listdir(file_dir_TC)
    nonTC_list = os.listdir(file_dir_nonTC)
    for file in TC_list[:len(nonTC_list)]:
        name = file.split('_')
        if name[0] == label_pos:
            TC.append(file_dir_TC + file)
            label_TC.append(1)
    for file in nonTC_list:
        name = file.split('_')
        if name[0] == label_neg:
            nonTC.append(file_dir_nonTC + file)
            label_nonTC.append(0)
    print("There are %d TC\nThere are %d nonTC" % (len(TC), len(nonTC)))
 
    # shuffle
    image_list = np.hstack((TC, nonTC))
    label_list = np.hstack((label_TC, label_nonTC))
    temp = np.array([image_list, label_list])
    temp = temp.transpose()    
    np.random.shuffle(temp)
 
    image_list = list(temp[:, 0])
    label_list = list(temp[:, 1])
    label_list = [int(i) for i in label_list]
 
    return image_list, label_list
 
# img_list,label_list = get_files(file_dir)
 
# batch
def get_batch(image, label, image_W, image_H, batch_size, capacity):   
    image = tf.cast(image, tf.string)
    label = tf.cast(label, tf.int32)
 
    # queue
    input_queue = tf.train.slice_input_producer([image, label])
 
    image_contents = tf.read_file(input_queue[0])
    label = input_queue[1]
    image = tf.image.decode_jpeg(image_contents, channels=1)
 
    # resize
    image = tf.image.resize_images(image, [image_H, image_W], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    image = tf.cast(image, tf.float32)
    # image = tf.image.per_image_standardization(image)  
    image_batch, label_batch = tf.train.batch([image, label],
                                              batch_size=batch_size,
                                              num_threads=64,  
                                              capacity=capacity)
  
    return image_batch, label_batch

In [ ]:
import os
import numpy as np
import tensorflow as tf
tf.reset_default_graph()
N_CLASSES = 2 
IMG_W = 64  # resize
IMG_H = 64
BATCH_SIZE = 64
CAPACITY = 2000
MAX_STEP = 20000000 
learning_rate = 0.0005 
 
train_dir = './train_attention_resnet_Residual/'
logs_train_dir = './train_attention_resnet_Residual/'

file_dir_TC='/home/ubuntu/data/TC/'
file_dir_nonTC='/home/ubuntu/data/nonTC/'
file_dir_valTC='/home/ubuntu/data/valTC_cp/'
file_dir_valnonTC='/home/ubuntu/data/valnonTC/'

train, train_label = get_files(file_dir_TC,file_dir_nonTC,'TC','nonTC')
valid, valid_label = get_files(file_dir_valTC,file_dir_valnonTC,'valTC','valnonTC')
train_batch_op,train_label_batch_op=get_batch(train,
                                train_label,
                                IMG_W,
                                IMG_H,
                                BATCH_SIZE,
                                CAPACITY)

valid_batch_op,valid_label_batch_op=get_batch(valid,
                                valid_label,
                                IMG_W,
                                IMG_H,
                                BATCH_SIZE,
                                CAPACITY)
x = tf.placeholder(tf.float32, [BATCH_SIZE,IMG_W,IMG_H,1])
y = tf.placeholder(tf.float32, [BATCH_SIZE])
y = tf.cast(y,tf.int64)

train_logits = res_attention_inference(x)
train_loss = losses(train_logits, y, 'train')
train_op = trainning(train_loss, learning_rate)
train_acc = evaluation(train_logits, y, 'train')
train_recall, train_precision = recall_precision(train_logits, y, 'train')

valid_loss = losses(train_logits, y, 'valid')
valid_acc = evaluation(train_logits, y, 'valid')
valid_recall, valid_precision = recall_precision(train_logits, y, 'valid')

summary_op = tf.summary.merge_all() 
 
sess = tf.Session()

train_writer = tf.summary.FileWriter(logs_train_dir, sess.graph)

saver = tf.train.Saver()

sess.run(tf.global_variables_initializer())
 

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)
 
try:
    print('----------training start---------')
    train_result=[]
    valid_result=[]
    for step in np.arange(MAX_STEP):
        train_batch,train_label_batch = sess.run([train_batch_op,train_label_batch_op])
        if coord.should_stop():
                break
       
        _, tra_loss, tra_acc, tra_recall, tra_precision= sess.run([train_op, train_loss, train_acc, train_recall, train_precision],feed_dict={x:train_batch,y:train_label_batch})
        summary_str = sess.run(summary_op,feed_dict={x:train_batch,y:train_label_batch})
        train_writer.add_summary(summary_str, step)
        train_result.append([step, tra_loss, tra_acc*100.0, tra_recall*100.0, tra_precision*100.0])
        if step % 50 == 0:
            print('Step %d, train loss = %.2f, train accuracy = %.2f%%, train recall = %.2f%%, train precision = %.2f%%' %(step, tra_loss, tra_acc*100.0, tra_recall*100.0, tra_precision*100.0))
            
            summary = tf.Summary()
            summary.value.add(tag='tra_loss', simple_value=tra_loss)
            summary.value.add(tag='tra_acc', simple_value=tra_acc)
            summary.value.add(tag='tra_recall', simple_value=tra_recall)
            summary.value.add(tag='tra_precision', simple_value=tra_precision)
            train_writer.add_summary(summary, step)
        if step % 500 == 0:
            ckpt=tf.train.get_checkpoint_state('./train_attention_resnet_Residual/')
#             print(ckpt)
            if ckpt and ckpt.all_model_checkpoint_paths:
                valid_batch,valid_label_batch = sess.run([valid_batch_op,valid_label_batch_op])
                saver.restore(sess,ckpt.model_checkpoint_path)
                val_loss, val_acc, val_recall, val_precision = sess.run([valid_loss, valid_acc, valid_recall, valid_precision], feed_dict={x:valid_batch,y:valid_label_batch})
                valid_result.append([step, val_loss, val_acc*100.0, val_recall*100.0, val_precision*100.0])
                print('*********************')
                print('Step %d, valid loss = %.2f, valid accuracy = %.2f%%, valid recall = %.2f%%, valid precision = %.2f%%' %(step, val_loss, val_acc*100.0, val_recall*100.0, val_precision*100.0))
                print('*********************')
                summary = tf.Summary()
                summary.value.add(tag='val_loss', simple_value=val_loss)
                summary.value.add(tag='val_acc', simple_value=val_acc)
                summary.value.add(tag='val_recall', simple_value=val_recall)
                summary.value.add(tag='val_precision', simple_value=val_precision)
                train_writer.add_summary(summary, step)
#                 summary_str = sess.run(summary_op,feed_dict={x:valid_batch,y:valid_label_batch})
#                 train_writer.add_summary(summary_str, step)
            
      
        if step % 500 == 0 or (step + 1) == MAX_STEP:
            checkpoint_path = os.path.join(logs_train_dir, 'model.ckpt')
            saver.save(sess, checkpoint_path, global_step=step)

except tf.errors.OutOfRangeError:
    print('Done training -- epoch limit reached')

finally:
    coord.request_stop()

There are 1737956 TC
There are 1737956 nonTC
There are 14355 TC
There are 434488 nonTC
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating: